<a href="https://colab.research.google.com/github/kaifkh20/nn_z_to_h/blob/main/makemore_backprop_ninja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving names(1).txt to names(1).txt


In [ ]:
import torch
import torch.nn.functional as F
words = open("names(1).txt").read().splitlines()

In [ ]:
#character mapping
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)

In [ ]:
#building the dataset

block_size = 3 #context length
X,Y = [],[]

for w in words:
  # print(w)
  context = [0] * block_size #pad it with ...
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    # print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix]
X = torch.tensor(X)
Y = torch.tensor(Y)

In [ ]:
#build training, dev and test dataset

def build_dataset(words):
  block_size = 3 #context length
  X,Y = [],[]

  for w in words:
    # print(w)
    context = [0] * block_size #pad it with ...
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      # print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix]
  X = torch.tensor(X)
  Y = torch.tensor(Y)

  return X,Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])


In [ ]:
#utitlity function to compare our gradient to pytorch's gradient
def cmp(s,dt,t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt,t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(
      f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}'
  )

In [ ]:
#MLP revisited

n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of MLP

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size,n_embd),generator=g)
W1 = torch.randn((n_embd*block_size,n_hidden),generator=g)*0.1
b1 = torch.randn(n_hidden,generator=g)*0.01
W2 = torch.randn((n_hidden,vocab_size),generator=g)*0.01
b2 = torch.randn(vocab_size,generator=g)*0

bngain = torch.ones((1,n_hidden))*0.1
bnbias = torch.zeros((1,n_hidden))
bnmean_running = torch.ones((1,n_hidden))
bnstd_running = torch.ones((1,n_hidden))
parameters = [C,W1,b1,b2,W2,bngain,bnbias]
print(sum(p.nelement() for p in parameters)) #parameters in total
for p in parameters:
  p.requires_grad = True

4137


In [ ]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time
batch_size = 32
n = batch_size # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y
emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.2961, grad_fn=<NegBackward0>)

In [31]:
# Exercise 1: backprop through the whole thing manually,
# backpropagating through exactly all of the variables
# as they are defined in the forward pass above, one by one

# -----------------
# YOUR CODE HERE :)
# -----------------

# !---------------------INTUTION FOR THIS -----------------!
#Example loss = -(a+b+c)/3
#        dloss/a = -1/3
# In case of dloss/dlogprobs = (logprobsi1 + logprobsi2 + .... + logprobsin / n) / logprobsi =
# so it will be for each i sum(-1/n,size_of_logprobs)

dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0/n #this just fills the Yb index position with -1/n since these only participate in the derivation

dprobs = (1.0/probs) * dlogprobs

dcounts_sum_inv = (dprobs * counts).sum(1,keepdim=True) #why this .sum() counts is of size (3,1) and (3,3) of dprobs
#so if we do * then pytorch does is (3,3) which we dont need so we sum along columns to get (3,1)

dcounts = counts_sum_inv * dprobs

dcounts_sum = (dcounts_sum_inv * (-counts_sum**-2))

dcounts += torch.ones_like(counts) * dcounts_sum

dnorm_logits = dcounts * counts

dlogits = dnorm_logits.clone()

dlogit_maxes = (-dnorm_logits).sum(1,keepdim=True)

dlogits += F.one_hot(logits.max(1).indices,num_classes=logits.shape[1]) * dlogit_maxes

dh = dlogits @ W2.T

cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
# cmp('W2', dW2, W2)
# cmp('b2', db2, b2)
# cmp('hpreact', dhpreact, hpreact)
# cmp('bngain', dbngain, bngain)
# cmp('bnbias', dbnbias, bnbias)
# cmp('bnraw', dbnraw, bnraw)
# cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
# cmp('bnvar', dbnvar, bnvar)
# cmp('bndiff2', dbndiff2, bndiff2)
# cmp('bndiff', dbndiff, bndiff)
# cmp('bnmeani', dbnmeani, bnmeani)
# cmp('hprebn', dhprebn, hprebn)
# cmp('embcat', dembcat, embcat)
# cmp('W1', dW1, W1)
# cmp('b1', db1, b1)
# cmp('emb', demb, emb)
# cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
